## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [2]:
nodes = json.load(open('new_nodes.json', 'r'))
roads = json.load(open('new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2239
Num roads 10138


Create two maps as lookup tables for the nodes

In [3]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [4]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.55, 45.59)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [5]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [6]:
# pool of end points 
# define a random number of destinations (e.g. from 8 to 12)
num_dest = random.randrange(8, 12, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 1145, 'coordinates': (12.02, 45.51)}, {'id': 844, 'coordinates': (12.16, 45.56)}, {'id': 799, 'coordinates': (12.75, 45.63)}, {'id': 1711, 'coordinates': (11.78, 45.24)}, {'id': 41, 'coordinates': (12.44, 45.55)}, {'id': 9, 'coordinates': (12.55, 45.59)}, {'id': 1577, 'coordinates': (12.66, 45.6)}, {'id': 25, 'coordinates': (12.25, 45.49)}]


In [7]:
def generate_trips(n):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    rand_s = random.randrange((len(node_coords)))
    id_start = list(node_coords.keys())[rand_s]
    coord_start = list(node_coords.values())[rand_s]
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    # end point
    rand_d = random.randrange(num_dest)
    dest = pool_dest[rand_d]
    trips_dest.append(dest)

  return trips_start, trips_dest


In [8]:
def generate_schedules(n):
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   6 : 0.2,
                  8 : 0.1,
                  10 : 0.07,
                  12 : 0.1,
                  14 : 0.07,
                  16 : 0.15,
                  18 : 0.15,
                  20 : 0.05,
                  22 : 0.05,
                  0 : 0.01,
                  2 : 0.01,
                  4 : 0.04 }

  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append(str(random.choices(hours, prob)[0]) + ":00")

  return times

In [9]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, hour, start_point, end_point):
    self.id = id
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Rider ( id = " + str(self.id) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [10]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(90, 150, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers)
  trips_start_riders, trips_dest_riders = generate_trips(n_riders)
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers
  ratings = generate_ratings(n_drivers)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [11]:
drivers, riders = generate_data()

for i in range(5) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(5) :
  print(riders[i])


N_drivers:  40
N_riders:  90
Driver ( id = 0, rating = 4.0, hour = 8:00, start = {'id': 732, 'coordinates': (11.97, 45.49)}, dest = {'id': 9, 'coordinates': (12.55, 45.59)})
Driver ( id = 1, rating = 3.5, hour = 8:00, start = {'id': 622, 'coordinates': (12.25, 45.41)}, dest = {'id': 25, 'coordinates': (12.25, 45.49)})
Driver ( id = 2, rating = 3.5, hour = 8:00, start = {'id': 1322, 'coordinates': (11.87, 45.21)}, dest = {'id': 1145, 'coordinates': (12.02, 45.51)})
Driver ( id = 3, rating = 4.0, hour = 18:00, start = {'id': 426, 'coordinates': (12.4, 45.61)}, dest = {'id': 844, 'coordinates': (12.16, 45.56)})
Driver ( id = 4, rating = 4.0, hour = 14:00, start = {'id': 145, 'coordinates': (12.44, 45.45)}, dest = {'id': 9, 'coordinates': (12.55, 45.59)})


***********************************************************************


Rider ( id = 0, hour = 12:00, start = {'id': 2227, 'coordinates': (12.03, 45.44)}, dest = {'id': 844, 'coordinates': (12.16, 45.56)})
Rider ( id = 1, hour = 18:00

In [13]:
# compute distance between two nodes

def compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2):
  x_coord_1 = x_coord_1 * 111.32
  x_coord_2 = x_coord_2 * 111.32
  y_coord_1 = 40075 * math.cos(x_coord_1)/360 * y_coord_1;
  y_coord_2 = 40075 * math.cos(x_coord_2)/360 * y_coord_2;
  dist = math.sqrt((y_coord_1 - y_coord_2)**2 + (x_coord_1 - x_coord_2)**2)
  return dist

In [28]:
# use of networkx, creation of graph

import networkx as nx
# use of networkx

G = nx.Graph()

G.add_nodes_from(range(len(nodes)))
for road in roads :
  n1 = int(road.get("p1"))
  n2 = int(road.get("p2"))
  x_coord_1 = float(nodes[n1].get('coordinates')[0])
  y_coord_1 = float(nodes[n1].get('coordinates')[1])
  x_coord_2 = float(nodes[n2].get('coordinates')[0])
  y_coord_2 = float(nodes[n2].get('coordinates')[1])
  dist = compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2)

  G.add_edge(n1, n2, weight=dist)



EdgeView([(0, 1), (0, 2), (1, 47), (1, 87), (1, 1281), (1, 1821), (2, 141), (2, 148), (3, 4), (3, 428), (3, 55), (3, 56), (4, 1285), (5, 1256), (5, 6), (5, 1233), (5, 1415), (5, 1413), (5, 2121), (5, 2066), (5, 1732), (6, 1254), (6, 1273), (6, 2062), (6, 2121), (7, 8), (7, 10), (7, 1207), (7, 1441), (8, 618), (8, 619), (8, 1443), (9, 769), (9, 10), (9, 1015), (9, 1017), (9, 981), (9, 1246), (10, 862), (10, 1207), (10, 1063), (10, 1442), (10, 1620), (11, 13), (11, 12), (11, 1276), (11, 1275), (11, 1278), (12, 1253), (13, 1252), (13, 14), (13, 1249), (13, 2133), (14, 1252), (14, 1230), (14, 2029), (15, 16), (15, 369), (15, 1894), (15, 2214), (16, 28), (16, 33), (16, 1994), (17, 23), (17, 32), (17, 35), (17, 82), (18, 19), (18, 537), (18, 102), (18, 317), (18, 99), (19, 102), (19, 613), (19, 692), (19, 624), (19, 1506), (20, 21), (20, 44), (20, 45), (20, 26), (20, 236), (21, 22), (21, 45), (21, 60), (21, 24), (22, 88), (22, 83), (22, 45), (22, 2068), (22, 60), (23, 2068), (23, 88), (23, 3

In [35]:
# use of Dijkstra for shortest path

drivers_path = []
riders_path = []

for driver in drivers:
  spath = nx.dijkstra_path(G, driver.start_point.get('id'), driver.end_point.get('id'))
  drivers_path.append({'id_driver': driver.id, 'shortest_path' : spath})

for rider in riders:
  spath = nx.dijkstra_path(G, rider.start_point.get('id'), rider.end_point.get('id'))
  riders_path.append({'id_rider': rider.id, 'shortest_path' : spath})

print(drivers_path)
print(riders_path)

[{'id_driver': 0, 'shortest_path': [732, 1112, 1113, 923, 789, 723, 725, 396, 397, 1152, 276, 788, 787, 888, 107, 889, 783, 514, 508, 312, 523, 510, 309, 314, 161, 162, 164, 173, 170, 171, 174, 1930, 1929, 1207, 10, 9]}, {'id_driver': 1, 'shortest_path': [622, 623, 624, 1506, 102, 317, 120, 300, 30, 122, 237, 227, 25]}, {'id_driver': 2, 'shortest_path': [1322, 1323, 1328, 1327, 1335, 1338, 1296, 1316, 880, 1309, 1313, 1320, 992, 991, 482, 490, 487, 486, 115, 69, 68, 70, 397, 1151, 342, 810, 811, 1456, 884, 885, 1145]}, {'id_driver': 3, 'shortest_path': [426, 164, 162, 161, 314, 309, 510, 523, 312, 508, 514, 783, 844]}, {'id_driver': 4, 'shortest_path': [145, 144, 146, 1282, 379, 140, 169, 378, 377, 148, 2, 0, 1, 47, 176, 165, 294, 1290, 1289, 218, 1216, 1286, 49, 48, 50, 1249, 13, 14, 1230, 1250, 1251, 1434, 1279, 1019, 769, 9]}, {'id_driver': 5, 'shortest_path': [1367, 1369, 1760, 878, 877, 1759, 2020, 1776, 1773, 1772, 1771, 2067, 1090, 677, 563, 562, 895, 804, 480, 149, 436, 800, 43

In [ ]:
# create file input.dat (obsolete (?))
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()